In [ ]:
# This cell is NOT editable. Overwrite variables on your own discretion.
# Any changes other than the script code will NOT BE SAVED!
# All cells are assumed to be script code cells, unless explictly tagged as 'o9_ignore'

In [ ]:
_sellOutData = "Select (&CWVAndScenarios * [Region].[Region L3] * [Region].[Region L2] * &RGMPlanningRegions *  &RGMChannels * &RGMAccounts * [Time].[Week] * [Item].[RGM1] * [Item].[RGM2] * [Item].[RGM7] * [Item].[RGM5] * [Item].[RGM6] * [Item].[PPG]) on row, ({Measure.[RGM %ACV], Measure.[RGM Promo Simulation Sales], Measure.[RGM Value Sales], Measure.[RGM Promo Simulation Base PTC]}) on column;"
_tradePromoCalendarData = "Select (&CWVAndScenarios * &PromotionInitiatives * [Region].[Region L3] * [Region].[Region L2] * &RGMPlanningRegions *  &RGMChannels * &RGMAccounts * &ClientPPGs * [Time].[Week]) on row, ({Measure.[RGM Initiative Week Sellout Start Date], Measure.[RGM Initiative Week Sellout End Date], Measure.[RGM Initiative Week Promo Mechanic], Measure.[RGM Promo Simulation Initiative Base PTC], Measure.[RGM Promo Simulation Initiative Promo PTC], Measure.[RGM Promo Simulation Initiative Base PTR], Measure.[RGM Promo Simulation Initiative Promo PTR], Measure.[RGM Initiative Week Promo Discount], Measure.[RGM Initiative Week Display Type]}) on column where {~isnull (Measure.[RGM Promo Simulation Initiative Base PTC])};"
_timeData = "select ([Time].[Planning Year] * [Time].[Planning Month] * [Time].[Week] * [Time].[Week Name] * [Time].[Day]) on row, () on column include memberproperties { [Time].[Planning Year] , [Key] } { [Time].[Planning Month], [Key] } { [Time].[Week], [Key] } { [Time].[Day], [Key] };"
_inputConfigParameter1 = "Select (&CWVAndScenarios) on row,  ({Measure.[RGM Historical Years], Measure.[RGM Seasonality Years], Measure.[RGM Learning Rate], Measure.[RGM ML Algorithms], Measure.[RGM Model Run Type],  Measure.[RGM Negative Correlation Threshold For Cannibalization], Measure.[RGM Percentage Change In Amplitude],  Measure.[RGM Positive Correlation Threshold For Halo], Measure.[RGM Seasonality Time Bucket]}) on column;"
_inputConfigParameter2 = "Select (&CWVAndScenarios * [Region].[Region L3] * [Item].[RGM5] * Item.[RGM6]) on row,  ({Measure.[RGM Discount Pct Threshold], Measure.[RGM Pantry Decay Rate]}) on column;"
_auditDataFrame = "Select (&CwvandScenarios * [Sequence].[RGM Series] * [Time].[Day] ) on row,  ({Measure.[RGM ML Algorithm History], Measure.[RGM Model Run Data End Date History], Measure.[RGM Model Run Data Start Date History], Measure.[RGM Model Run Timestamp History], Measure.[RGM Model Run Type History], Measure.[RGM Model Runtime History], Measure.[RGM Plugin Name History]}) on column;"
_tenantCurrDay = "Select (&AllDays.Filter(#.[Day$IsCurrent]));"
_tenantSequenceID = "Select (&CwvAndScenarios) on row,  ({Measure.[RGM Model Run Sequence ID1], Measure.[RGM Model Run Sequence ID2], Measure.[RGM Model Run Sequence ID3]}) on column;"


# Initialize the O9DataLake with the input parameters and dataframes
# Data can be accessed with O9DataLake.get(<Input Name>)
# Overwritten values will not be reflected in the O9DataLake after initialization

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
sellOutData = O9DataLake.register("sellOutData",DataSource.LS, ResourceType.IBPL, _sellOutData)
tradePromoCalendarData = O9DataLake.register("tradePromoCalendarData",DataSource.LS, ResourceType.IBPL, _tradePromoCalendarData)
timeData = O9DataLake.register("timeData",DataSource.LS, ResourceType.IBPL, _timeData)
inputConfigParameter1 = O9DataLake.register("inputConfigParameter1",DataSource.LS, ResourceType.IBPL, _inputConfigParameter1)
inputConfigParameter2 = O9DataLake.register("inputConfigParameter2",DataSource.LS, ResourceType.IBPL, _inputConfigParameter2)
auditDataFrame = O9DataLake.register("auditDataFrame",DataSource.LS, ResourceType.IBPL, _auditDataFrame)
tenantCurrDay = O9DataLake.register("tenantCurrDay",DataSource.LS, ResourceType.IBPL, _tenantCurrDay)
tenantSequenceID = O9DataLake.register("tenantSequenceID",DataSource.LS, ResourceType.IBPL, _tenantSequenceID)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# ## 2. RGM - Data Pre-processing (2. RGM Trade Promo Modelling - Preprocessing)

# In[1]:


# Import all the necessary libraries.
import os
import pandas as pd
import numpy as np
import datetime, time
import warnings
warnings.filterwarnings('ignore')
logger = logging.getLogger('o9_logger')


# In[2]:


startTime = time.time()
logger.debug("Started running pre-processing code for promo model...")


# In[3]:


#Enhancement log:
#Config2.0:
#1.UserStory-212123: Incorporate the Sales Domain Dimension Breakup / regraning dimensions name with new Name


# In[4]:


# Read all the input data.
# sellOutData = pd.read_csv("Sellout DS Tenant.csv")
# tradePromoCalendarData = pd.read_csv("Promo Initiative Calendar DS Tenant.csv")
# #tradePromoCalendarData = pd.read_csv("Promo Initiative Calendar DS Tenant-empty.csv") #UserStory:206082-(Config 2.0),Enhancing Pricing Model to work with & without Trade Promo Calendar Data
# timeData = pd.read_csv("Time Master Tenant.csv")
# inputConfigParameter1 = pd.read_csv("Parameter 1 Tenant.csv")
# inputConfigParameter2 = pd.read_csv("Parameter 2 Tenant.csv")
# auditDataFrame = pd.read_csv('auditTableDataOutputFinal.csv')
# tenantCurrDay = pd.read_csv("Dimension.AllDays.csv")
# tenantSequenceID = pd.read_csv("sequenceIDDataOutput_Plugin0.csv")
# # holidayData = pd.read_csv("Fact.Holiday.csv")
# # powerDateData = pd.read_csv("Fact.RGMPowerDates.csv")
# #
# # weatherDataDailyLevel = pd.read_csv("Fact.WeatherData.csv")
# # googleMobilityDataWeeklyLevel = pd.read_csv("Fact.GoogleMobilityData.csv")
# # unemploymentDataMonthlyLevel = pd.read_csv("Fact.UnemploymentData.csv")
# # consumerSentimentDataMonthlyLevel = pd.read_csv("Fact.ConsumerSentimentData.csv")
# #
# # loyaltyData = pd.read_csv("Fact.LoyaltyDataNew.csv")


# In[5]:


logger.debug("For sellOutData, shape is " + str(sellOutData.shape))
logger.debug("For tradePromoCalendarData, shape is " + str(tradePromoCalendarData.shape))
logger.debug("For timeData, shape is " + str(timeData.shape))
logger.debug("For inputConfigParameter1, shape is " + str(inputConfigParameter1.shape))
logger.debug("For inputConfigParameter2, shape is " + str(inputConfigParameter2.shape))
logger.debug("For auditDataFrame, shape is " + str(auditDataFrame.shape))
logger.debug("For tenantCurrDay, shape is " + str(tenantCurrDay.shape))
logger.debug("For tenantSequenceID, shape is " + str(tenantSequenceID.shape))
logger.debug("Successfully read the input data.")


# In[6]:


# Variable Config For Dynamic Name

# Sell Out Data Column Names
versionVariable = 'Version.[Version Name]'
# UserStory-212123
countryVariable='Region.[Region L3]' # Contains the Country info
regionVariable = 'Region.[Region L2]' # Contains the Region info
locationVariable = 'Region.[Planning Region]' # Contains the store/store cluster/region data
channelVariable = 'Channel.[Channel]' # Contains the Channel info
customerGroupVariable = 'Account.[Account]' # Name of customer group column/Contains the Account info
locationCountryVariable = 'Region.[Region L3]_duplicate' # (Region.[Region L3] will now contain the country info, to avoid the code change we are create the duplicate column )
#
rgm1Variable = 'Item.[RGM1]' # Self vs competitors column
rgm2Variable = 'Item.[RGM2]' # Manufactur column
rgm5Variable = 'Item.[RGM5]' # Category column
rgm6Variable = 'Item.[RGM6]' # Sub-category column
rgm7Variable = 'Item.[RGM7]' # Sub-Category + Brand column
ppgVariable = 'Item.[PPG]' # Name of PPG column
unitSizeVariable = 'Item.[Unit Size]' # Unit size column
packSizeVariable = 'Item.[Pack Size]' # Pack size column
acvVariable = 'RGM %ACV' # Name of weighted distribution column
valueSalesVariable = 'RGM Value Sales' # Name of revenue column
dependentVariable = 'RGM Promo Simulation Sales' # Name of dependent variable for the modelling
basePriceVariable = 'RGM Promo Simulation Base PTC' # Name of column to be taken for base price for modelling

# Time Data Column Names
weekVariable = 'Time.[Week]' # Name of week granularity column
dayVariable = 'Time.[Day]' # Name of day granularity column
planningMonthVariable = 'Time.[Planning Month]' # Name of planning month column
weekNameVariable = 'Time.[Week Name]' # Name of week column

# Promo Data Column Names
promoMechanicColumnName = 'RGM Initiative Week Promo Mechanic' # Name of column from trade promo calendar data which we should use for promo mechanic.
promoDisplayColumnName = 'RGM Initiative Week Display Type' # Name of column from trade promo calendar data which we should use for promo display.
sellOutStartDateColumnName = 'RGM Initiative Week Sellout Start Date' # Name of column for sell out start date.
sellOutEndDateColumnName = 'RGM Initiative Week Sellout End Date' # Name of column for sell out end date.

# Defined column names for Audit Table:
seriesVariable = 'Sequence.[RGM Series]'
modelRunTypeVariableName = 'RGM Model Run Type History' 
pluginVariableName = 'RGM Plugin Name History'
machineLearningAlgorithmVariableName = 'RGM ML Algorithm History'
dataStartDateVariableName = 'RGM Model Run Data Start Date History'
dataEndDateVariableName = 'RGM Model Run Data End Date History'
pluginExecutionTimeStampVariableName = 'RGM Model Run Timestamp History'
pluginExecutionTimeVariableName = 'RGM Model Runtime History'

sequenceId = 'RGM Model Run Sequence ID1'

# Market Intel Data or External Data Column Names
marketIntelCountryVariable = 'Geo.[Country]'

# Name of columne that we are using for building model.
nameOfColumnForWhichBuildingModel = 'Item.[PPG]'

# It is either freshModelRun, refreshWithoutModelRun, refreshWithModelRun 
modelRunType = inputConfigParameter1['RGM Model Run Type'][0]

# Choose the name of model to be used for Modelling i.e. from "linearRegression", "constrainedLinearRegression" and "lsqLinear"
modelName = inputConfigParameter1['RGM ML Algorithms'][0]


# In[7]:


# UserStory:212123:Due to regraning of sales domain, locationCountryVariable is same as countryVariable,i.e Region.[Region L3] will now contain the country info, to avoid the code change we are create the duplicate column
# in the dataframe where we are using locationCountryVariable.
sellOutData[locationCountryVariable]=sellOutData[countryVariable]
tradePromoCalendarData[locationCountryVariable]=tradePromoCalendarData[countryVariable]


# In[8]:


# This is used for the configuration of external data to be used for modeling for individual countries.
# e.g: variableName = {countryName: [listOfExternalDataFrames]}

# Pass all the daily level external factors data frames in the form of list in the below shown dictionary.
dictOfDailyLevelExternalData = {'USA':[], 'UK':[]}

# Pass all the weekly level external factors data frames in the form of list in the below shown dictionary.
dictOfWeeklyLevelExternalData = {'USA':[], 'UK':[]}

# Pass all the monthly level external factors data frames in the form of list in the below shown dictionary.
dictOfMonthlyLevelExternalData = {'USA':[], 'UK':[]}


# In[9]:


# This is like a configuration where user can input or change predefined values as per there requirement.

# Seasonality Index should be calculate at 'Week' or 'Month' level depending on data granularity.
seasonalityTimeGranularity = inputConfigParameter1['RGM Seasonality Time Bucket'][0]

# Years for which we need to build the model.
try:
    listOfModellingYear = [int(i.replace(' ','')) for i in inputConfigParameter1['RGM Historical Years'][0].split(',') ]
except:
    listOfModellingYear = [int(inputConfigParameter1['RGM Historical Years'])]

# Year that we need to consider for seasonality calculation.
try:
    listOfSeasonalityYear = [int(i.replace(' ','')) for i in inputConfigParameter1['RGM Seasonality Years'][0].split(',') ]
except:
    listOfSeasonalityYear = [int(inputConfigParameter1['RGM Seasonality Years'])]

# Which promo method should be used? Don't keep both the method as 'Yes' at a time.
promoMethod1 = 'Yes'  # It will create individual column for Display & Mechanics. 
promoMethod2 = 'No'   # It will create single column for Display & Mechanic by taking their cross product.

# Which method should be used to merge sellOut and tradePromoCalendar data. "method1" or "method2"
preparePromoDataMethod = "method1"


# In[10]:


# Input Variables Config:

# List Of input variables for model.
listOfPromoMechanicsForModel = list(tradePromoCalendarData[promoMechanicColumnName].unique())
listOfPromoDisplayForModel = list(tradePromoCalendarData[promoDisplayColumnName].unique())
listOfInputVariablesForModel = ['Base Price', 'Discount Index', 'Weighted Distribution', 'Pantry Loading', 'Trendline', 'Seasonality Index']
listOfInputVariablesForModel.extend(listOfPromoMechanicsForModel)
listOfInputVariablesForModel.extend(listOfPromoDisplayForModel)

# List Of input variables for model for which we need to take the log.
listOfInputVariablesWithLogForModel = ['Base Price', 'Discount Index', 'Weighted Distribution', 'Pantry Loading', 'Trendline', 'Seasonality Index']


# In[11]:


logger.debug("Config setup is done successfully.")


# In[12]:


# Create Date, Year, Month & Week columns.
timeDataOriginal = timeData.copy()
timeDataOriginal = timeDataOriginal[[weekVariable, weekNameVariable]].drop_duplicates().reset_index(drop=True)
timeDataOriginal["Year"] = pd.DatetimeIndex(timeDataOriginal[weekVariable]).year
timeDataOriginal["Month"] = pd.DatetimeIndex(timeDataOriginal[weekVariable]).month
timeDataOriginal['Week'] = timeDataOriginal[weekNameVariable].apply(lambda x: int(x[1:]))
timeDataOriginal = timeDataOriginal[[weekVariable, 'Year', 'Month', 'Week']]

timeData = timeData[[dayVariable, weekNameVariable]]
timeData['Week'] = timeData[weekNameVariable].apply(lambda x: int(x[1:]))
timeData = timeData[[dayVariable, 'Week']].rename(columns={dayVariable:weekVariable})

sellOutData["Date"] = pd.DatetimeIndex(sellOutData[weekVariable])
sellOutData["Year"] = pd.DatetimeIndex(sellOutData[weekVariable]).year
sellOutData["Month"] = pd.DatetimeIndex(sellOutData[weekVariable]).month
sellOutData = sellOutData.merge(timeData, how='left', on=weekVariable)


# In[13]:


# timeData.to_csv('timeData.csv')
# timeDataOriginal.to_csv('timeDataOriginal.csv')


# In[14]:


# Create a granularity variable at which we want to aggregate the data for the modelling.
sellOutDataGranularityGroupBy = [versionVariable, countryVariable, regionVariable, locationCountryVariable, locationVariable, 
                                 channelVariable, customerGroupVariable, rgm1Variable, rgm2Variable, 
                                 rgm5Variable, rgm6Variable, rgm7Variable, ppgVariable, 'Year', 'Month', 'Week']


# In[15]:


# Filter out those rows which don't have any significant Volume and Value, sometimes you might get null.
sellOutData = sellOutData[(sellOutData[valueSalesVariable]!=0) | (sellOutData[dependentVariable]!=0)].reset_index(drop=True)
sellOutData = sellOutData[(sellOutData[valueSalesVariable].notna())| (sellOutData[dependentVariable].notna())].reset_index(drop=True)
logger.debug("For sellOutData, shape is " + str(sellOutData.shape))
#
# Filter out those rows which don't have any start or end date Value, sometimes you might get null.
tradePromoCalendarData = tradePromoCalendarData[(tradePromoCalendarData['RGM Initiative Week Sellout Start Date'].notna()) | (tradePromoCalendarData['RGM Initiative Week Sellout End Date'].notna())].reset_index(drop=True)
logger.debug("For tradePromoCalendarData, shape is " + str(tradePromoCalendarData.shape))
#
# Filter out the sell Out data for the year we are modelling for.
sellOutData = sellOutData[sellOutData['Year'].isin(listOfModellingYear)]


# In[16]:


# This method is used to fill missing dates in sell out data for each PPG on the basis of their Start and End Dates.
def fill_missing_rows_between_start_and_end_date_of_sellOut_data(dataFrameToBeFilled):
    timeDummyData = pd.DataFrame(dataFrameToBeFilled[weekVariable].unique(), columns=[weekVariable]).merge(timeData, how='left', on=weekVariable)
    timeDummyData['Year'] = pd.DatetimeIndex(timeDummyData[weekVariable]).year
    timeDummyData['Month'] = pd.DatetimeIndex(timeDummyData[weekVariable]).month
    timeDummyData['YMW'] = timeDummyData['Year'].astype(str) + timeDummyData['Month'].astype(str) + timeDummyData['Week'].astype(str)
    timeDummyData.drop(columns=[weekVariable], inplace=True)
    timeDummyData = timeDummyData[['Year', 'Month', 'Week', 'YMW']]
    dataFrameToBeFilled['Granularity'] = dataFrameToBeFilled[versionVariable].astype(str) + dataFrameToBeFilled[countryVariable].astype(str) + dataFrameToBeFilled[regionVariable].astype(str) + dataFrameToBeFilled[locationVariable].astype(str) + dataFrameToBeFilled[locationCountryVariable].astype(str) + dataFrameToBeFilled[channelVariable].astype(str) + dataFrameToBeFilled[customerGroupVariable].astype(str) + dataFrameToBeFilled[rgm1Variable].astype(str) + dataFrameToBeFilled[rgm2Variable].astype(str) + dataFrameToBeFilled[rgm5Variable].astype(str) + dataFrameToBeFilled[rgm6Variable].astype(str) +  dataFrameToBeFilled[rgm7Variable].astype(str) + dataFrameToBeFilled[ppgVariable].astype(str)
    dataFrameToBeFilled['YMW'] = dataFrameToBeFilled['Year'].astype(str) + dataFrameToBeFilled['Month'].astype(str) + dataFrameToBeFilled['Week'].astype(str)
    fillMissingDate = pd.DataFrame()
    for i in dataFrameToBeFilled["Granularity"].unique():
        timeDummy = timeDummyData.copy()
        dataDummy = dataFrameToBeFilled[dataFrameToBeFilled["Granularity"] == i].reset_index(drop=True)
        startIndex = list(timeDummy['YMW']).index(dataDummy['YMW'][0])
        endIndex = list(timeDummy['YMW']).index(dataDummy['YMW'][len(dataDummy)-1])
        timeDummy = timeDummy[startIndex:endIndex+1].reset_index(drop=True)
        dataDummy = timeDummy.merge(dataDummy, how='left', on=['Year', 'Month', 'Week', 'YMW']).reset_index(drop=True).fillna(0)
        dataDummy.drop(columns=['YMW', 'Granularity'], inplace=True)
        dataDummy[versionVariable] = dataDummy[versionVariable][0]
        dataDummy[countryVariable] = dataDummy[countryVariable][0]
        dataDummy[regionVariable] = dataDummy[regionVariable][0]
        dataDummy[locationVariable] = dataDummy[locationVariable][0]
        dataDummy[locationCountryVariable] = dataDummy[locationCountryVariable][0]
        dataDummy[channelVariable] = dataDummy[channelVariable][0]
        dataDummy[customerGroupVariable] = dataDummy[customerGroupVariable][0]
        dataDummy[rgm1Variable] = dataDummy[rgm1Variable][0]
        dataDummy[rgm2Variable] = dataDummy[rgm2Variable][0]
        dataDummy[rgm5Variable] = dataDummy[rgm5Variable][0]
        dataDummy[rgm6Variable] = dataDummy[rgm6Variable][0]
        dataDummy[rgm7Variable] = dataDummy[rgm7Variable][0]
        dataDummy[ppgVariable] = dataDummy[ppgVariable][0]
        fillMissingDate = pd.concat([fillMissingDate, dataDummy], axis = 0).reset_index(drop=True)
    dataFrameToBeFilled = fillMissingDate.copy()
    return dataFrameToBeFilled


# In[17]:


# Calculating the selling price through Value and Volume columns using the formula mentioned below in the code.
def selling_price(sellOutData):
    sellOutData["Selling Price"] = sellOutData[valueSalesVariable]/sellOutData[dependentVariable]
    return sellOutData


# In[18]:


# Renaming basePriceVariable to "Base Price".
def base_price(sellOutData):
    sellOutData = selling_price(sellOutData)
    sellOutData['Granularity'] = sellOutData[versionVariable].astype(str) + sellOutData[countryVariable].astype(str) + sellOutData[regionVariable].astype(str) + sellOutData[locationVariable].astype(str) + sellOutData[locationCountryVariable].astype(str) + sellOutData[channelVariable].astype(str) + sellOutData[customerGroupVariable].astype(str) + sellOutData[rgm1Variable].astype(str) + sellOutData[rgm2Variable].astype(str) + sellOutData[rgm5Variable].astype(str) + sellOutData[rgm6Variable].astype(str) + sellOutData[rgm7Variable].astype(str) + sellOutData[ppgVariable].astype(str)
    salesData = sellOutData[sellOutDataGranularityGroupBy + [valueSalesVariable, dependentVariable, 'Granularity' ,'Selling Price', basePriceVariable]]
    salesData = salesData.rename(columns={basePriceVariable:'Base Price'})
    return salesData


# In[19]:


# This method is used to forward fill all the missing base price value.
def base_price_missing_values_treatment(salesData):
    salesData['Base Price'] = np.where(salesData['Base Price']==0, np.nan, salesData['Base Price'])
    salesData['Base Price'] = salesData['Base Price'].ffill(axis = 0)
    return salesData


# In[20]:


# Calculating %discount and discount index using the formula mentioned below in the code.
def discount_index(sellOutData):
    salesData = base_price(sellOutData)
    salesDataInputConfigParameter2CommonColumns = [i for i in inputConfigParameter2.columns if i in salesData]
    salesData = salesData.merge(inputConfigParameter2[salesDataInputConfigParameter2CommonColumns+['RGM Discount Pct Threshold']], how='left', on=salesDataInputConfigParameter2CommonColumns)
    salesData["Discount"] = ((salesData["Base Price"] - salesData["Selling Price"]) / salesData["Base Price"])
    salesData["Discount"] = np.where(salesData["Discount"]>=salesData['RGM Discount Pct Threshold'], salesData["Discount"], 0)
    salesData["Discount Index"] = ((salesData["Selling Price"]- salesData["Base Price"]) / salesData["Base Price"]) + 1
    salesData["Discount Index"] = np.where((salesData["Discount Index"] != 1) & ((1 - salesData["Discount Index"]) <= salesData['RGM Discount Pct Threshold']), 1, salesData["Discount Index"])
    salesData["Discount Index"] = salesData["Discount Index"].fillna(1)
    salesData = salesData.drop(columns='RGM Discount Pct Threshold')
    salesData = base_price_missing_values_treatment(salesData)
    salesData = salesData.reset_index(drop=True)
    return salesData


# In[22]:


# Renaming acvVariable to "Weighted Distribution".
def distribution(sellOutData):
    data = sellOutData.copy()
    data = data[sellOutDataGranularityGroupBy + [acvVariable]]
    data = data.rename(columns={acvVariable:'Weighted Distribution'})
    distributionData = data.copy()
    return distributionData


# In[23]:


# Retailers or store will increase their shelf capacity whenever there is a high Discount Index running on a products.
# We are setting pantry loading flag as 1 when Discount Index > 0 else it is 1 + decay rate * previous term.
def pantry_loading(salesData):
    salesDataInputConfigParameter2CommonColumns = [i for i in inputConfigParameter2.columns if i in salesData]
    salesData = salesData.merge(inputConfigParameter2[salesDataInputConfigParameter2CommonColumns+['RGM Pantry Decay Rate']], how='left', on=salesDataInputConfigParameter2CommonColumns)
    pantryDataMain = pd.DataFrame()
    for i in salesData["Granularity"].unique():
        pantryData = salesData[salesData["Granularity"] == i]
        pantryData = pantryData.reset_index(drop=True)
        pantryData["Pantry Loading"] = np.nan
        for j in range(0, len(pantryData)):
            if j == 0:
                pantryData["Pantry Loading"][j] = np.where(pantryData["Discount Index"][j]<0, 1, 1)
            if j>0:
                pantryData["Pantry Loading"][j] = np.where(pantryData["Discount Index"][j]==1, np.max([(2-pantryData["Discount Index"][j-1]), (pantryData["Pantry Loading"][j-1] ** pantryData['RGM Pantry Decay Rate'][j])]), 1)
        pantryData = pantryData.drop(columns='RGM Pantry Decay Rate')
        pantryDataMain = pd.concat([pantryDataMain, pantryData]).reset_index(drop=True)
    salesData = pantryDataMain.copy()
    return salesData


# In[24]:


# P2 = MAX(P1,IF(LEFT(B2,1)<>LEFT(B1,1),1,0)) where 'P' is price threshold while 'B' is baseprice.
def price_threshold(salesData):
    data = pd.DataFrame()
    for i in salesData["Granularity"].unique():
        data_1 = salesData[salesData["Granularity"] == i].reset_index(drop=True)
        data_1["Price Threshold"] = 0
        for j in range(1, len(data_1)):
            if int(str(data_1["Base Price"][j]).split('.')[0]) != int(str(data_1["Base Price"][j-1]).split('.')[0]):
                data_1["Price Threshold"][j] = max(1, data_1["Price Threshold"][j-1])
            else:
                data_1["Price Threshold"][j] = max(0, data_1["Price Threshold"][j-1])
        data = pd.concat([data, data_1], axis = 0).reset_index(drop=True)
    salesData = data.copy()
    return salesData


# In[25]:


# Merging trade promo calendar data with sellout data on the basis of sellout week fall under the sellout start and end date.
def merge_selloutdata_with_tradepromocalendardata_method1(sellOutData, tradePromoCalendarData, timeData):
    tradePromoCalendarData["Year"] = pd.DatetimeIndex(tradePromoCalendarData[weekVariable]).year
    tradePromoCalendarData["Month"] = pd.DatetimeIndex(tradePromoCalendarData[weekVariable]).month
    tradePromoCalendarData = tradePromoCalendarData.merge(timeData, how='left', on=weekVariable)
    tradePromoCalendarData['Display-Mechanic'] = tradePromoCalendarData[promoDisplayColumnName] + '-' + tradePromoCalendarData[promoMechanicColumnName]
    sellOutTradePromoCommonColumns = [i for i in sellOutData.columns if i in tradePromoCalendarData.columns] + ['Year', 'Month', 'Week']
    sellOutAndTradePromoData = sellOutData.merge(tradePromoCalendarData, how='left', on=sellOutTradePromoCommonColumns).fillna(0).reset_index(drop=True)
    return sellOutAndTradePromoData


# In[26]:


# Merging trade promo calendar data with sellout data on the basis of sellout week fall under the sellout start and end date.
def merge_selloutdata_with_tradepromocalendardata_method2(sellOutData, tradePromoCalendarData, timeData):
    timeData1 = timeData.copy()
    timeData1[weekVariable] = pd.DatetimeIndex(timeData1[weekVariable])
    tradePromoCalendarData = tradePromoCalendarData[(tradePromoCalendarData[promoMechanicColumnName].notnull()) | (tradePromoCalendarData[promoDisplayColumnName].notnull())].reset_index(drop=True)
    tradePromoCalendarDataColumnsOriginalList = list(tradePromoCalendarData.columns)
    tradePromoCalendarDataAfterColumnsDropList = tradePromoCalendarDataColumnsOriginalList.copy()
    tradePromoCalendarDataAfterColumnsDropList.remove(sellOutStartDateColumnName)
    tradePromoCalendarDataAfterColumnsDropList.remove(sellOutEndDateColumnName)
    timeData1.rename(columns={weekVariable:sellOutStartDateColumnName}, inplace=True)
    tradePromoCalendarData[sellOutStartDateColumnName] = pd.DatetimeIndex(tradePromoCalendarData[sellOutStartDateColumnName])
    tradePromoCalendarData['SO Start Year'] = pd.DatetimeIndex(tradePromoCalendarData[sellOutStartDateColumnName]).year
    tradePromoCalendarData['SO Start Month'] = pd.DatetimeIndex(tradePromoCalendarData[sellOutStartDateColumnName]).month
    tradePromoCalendarData = tradePromoCalendarData.merge(timeData1, how='left', on=sellOutStartDateColumnName)
    tradePromoCalendarData.rename(columns={'Week':'SO Start Week'}, inplace=True)
    timeData1.rename(columns={sellOutStartDateColumnName:sellOutEndDateColumnName}, inplace=True)
    tradePromoCalendarData[sellOutEndDateColumnName] = pd.DatetimeIndex(tradePromoCalendarData[sellOutEndDateColumnName])
    tradePromoCalendarData['SO End Year'] = pd.DatetimeIndex(tradePromoCalendarData[sellOutEndDateColumnName]).year
    tradePromoCalendarData['SO End Month'] = pd.DatetimeIndex(tradePromoCalendarData[sellOutEndDateColumnName]).month
    tradePromoCalendarData = tradePromoCalendarData.merge(timeData1, how='left', on=sellOutEndDateColumnName)
    tradePromoCalendarData.rename(columns={'Week':'SO End Week'}, inplace=True)
    tradePromoCalendarData['SO Year Difference'] = tradePromoCalendarData['SO End Year'] - tradePromoCalendarData['SO Start Year']
    tradePromoCalendarData['SO Month Difference'] = tradePromoCalendarData['SO End Month'] - tradePromoCalendarData['SO Start Month']
    tradePromoCalendarData['SO Week Difference'] = tradePromoCalendarData['SO End Week'] - tradePromoCalendarData['SO Start Week']
    maxWeek = tradePromoCalendarData.groupby(by='SO Start Year')['SO Start Week'].max().reset_index().rename(columns={'SO Start Week':'Maximum Week Number'})
    tradePromoCalendarData = tradePromoCalendarData.merge(maxWeek, how='left', on='SO Start Year').reset_index(drop=True)
    tradePromoCalendarData['SO Month Difference'] = np.where(tradePromoCalendarData['SO Month Difference'] < 0, 12 + tradePromoCalendarData['SO End Month'] - tradePromoCalendarData['SO Start Month'], tradePromoCalendarData['SO Month Difference'])
    tradePromoCalendarData['SO Week Difference'] = np.where(tradePromoCalendarData['SO Week Difference'] < 0, tradePromoCalendarData['Maximum Week Number'] + tradePromoCalendarData['SO End Week'] - tradePromoCalendarData['SO Start Week'], tradePromoCalendarData['SO Week Difference'])
    tradePromoCalendarDataDataFrame1 = tradePromoCalendarData[(tradePromoCalendarData['SO Week Difference']==0) | (tradePromoCalendarData['SO Week Difference']==1)].reset_index(drop=True)
    tradePromoCalendarDataDataFrame2 = tradePromoCalendarData[tradePromoCalendarData['SO Week Difference']>1].reset_index(drop=True)
    tradePromoCalendarDataDataFrame3 = pd.DataFrame()
    for i in range(0, len(tradePromoCalendarDataDataFrame2)):
        data = tradePromoCalendarDataDataFrame2[i:i+1].reset_index(drop=True)
        sellOutStartDateList = pd.date_range(start = data[sellOutStartDateColumnName][0], periods=data['SO Week Difference'][0], freq='W') 
        sellOutEndDate = data[sellOutEndDateColumnName][0]
        dateListDataFrame = pd.DataFrame({ sellOutEndDateColumnName:sellOutEndDate, 'Sell Out Start Date':sellOutStartDateList})
        dataDummy = dateListDataFrame.merge(data, how='left', on=sellOutEndDateColumnName)
        dataDummy[sellOutStartDateColumnName] = pd.DatetimeIndex(dataDummy['Sell Out Start Date'])
        dataDummy.drop(columns='Sell Out Start Date', inplace=True)
        dataDummy = dataDummy[data.columns]
        tradePromoCalendarDataDataFrame3 = pd.concat([tradePromoCalendarDataDataFrame3, dataDummy]).reset_index(drop=True)
    tradePromoCalendarData = pd.concat([tradePromoCalendarDataDataFrame1, tradePromoCalendarDataDataFrame3]).reset_index(drop=True)
    tradePromoCalendarDataMelt = pd.melt(tradePromoCalendarData, id_vars=tradePromoCalendarDataAfterColumnsDropList, value_vars=[sellOutStartDateColumnName, sellOutEndDateColumnName], var_name='Trade Promo Date Type', value_name='Trade Promo Date')
    tradePromoCalendarDataMelt['Display-Mechanic'] = tradePromoCalendarDataMelt[promoDisplayColumnName] + '-' + tradePromoCalendarDataMelt[promoMechanicColumnName]
    tradePromoCalendarDataMelt['Year'] = pd.DatetimeIndex(tradePromoCalendarDataMelt['Trade Promo Date']).year
    tradePromoCalendarDataMelt['Month'] = pd.DatetimeIndex(tradePromoCalendarDataMelt['Trade Promo Date']).month
    timeData1.rename(columns={sellOutEndDateColumnName:'Trade Promo Date'}, inplace=True)
    tradePromoCalendarDataMelt = tradePromoCalendarDataMelt.merge(timeData1, how='left', on='Trade Promo Date')
    tradePromoCalendarDataMelt.drop_duplicates(inplace=True)
    sellOutTradePromoCommonColumns = [i for i in sellOutData.columns if i in tradePromoCalendarData.columns] + ['Year', 'Month', 'Week']
    sellOutAndTradePromoData = sellOutData.merge(tradePromoCalendarDataMelt, how='left', on=sellOutTradePromoCommonColumns).fillna(0).reset_index(drop=True)
    return sellOutAndTradePromoData


# In[27]:


def prepare_promo_data(sellOutData, tradePromoCalendarData, timeData):
    if preparePromoDataMethod == "method1":
        sellOutAndTradePromoData = merge_selloutdata_with_tradepromocalendardata_method1(sellOutData, tradePromoCalendarData, timeData)
    elif preparePromoDataMethod == "method2":
        sellOutAndTradePromoData = merge_selloutdata_with_tradepromocalendardata_method2(sellOutData, tradePromoCalendarData, timeData)
    else:
        logger.debug("preparePromoDataMethod variable is missing.")
    sellOutAndTradePromoData = sellOutAndTradePromoData[(sellOutAndTradePromoData[promoMechanicColumnName]!=0) | (sellOutAndTradePromoData[promoDisplayColumnName]!=0)].reset_index(drop=True)
    sellOutAndTradePromoData['Intermediate Promo Flag'] = 1
    if promoMethod1 == 'Yes':
        promoMechanicData = sellOutAndTradePromoData.pivot_table(index=sellOutDataGranularityGroupBy, columns=promoMechanicColumnName, values='Intermediate Promo Flag', aggfunc=np.mean).reset_index()
        promoMechanicData = promoMechanicData.rename_axis(None, axis=1)
        promoDisplayData = sellOutAndTradePromoData.pivot_table(index=sellOutDataGranularityGroupBy, columns=promoDisplayColumnName, values='Intermediate Promo Flag', aggfunc=np.mean).reset_index()
        promoDisplayData = promoDisplayData.rename_axis(None, axis=1)
        promoMechanicAndDisplayData = promoDisplayData.merge(promoMechanicData, how='left', on=sellOutDataGranularityGroupBy).fillna(0)
    if promoMethod2 == 'Yes':
        promoMechanicAndDisplayData = sellOutAndTradePromoData.pivot_table(index=sellOutDataGranularityGroupBy, columns='Display-Mechanic', values='Intermediate Promo Flag', aggfunc=np.mean).reset_index()
        promoMechanicAndDisplayData = promoMechanicAndDisplayData.rename_axis(None, axis=1)
    return promoMechanicAndDisplayData


# In[28]:


# Trendline is a 45 degree trend line used to capture the sales trend over the period of time.
def trend_line(salesData):
    trendDataMain = pd.DataFrame()
    for i in salesData["Granularity"].unique():
        trendData = salesData[salesData["Granularity"] == i]
        trendData = trendData.reset_index(drop=True)
        trendData["Trendline"] = np.nan
        for j in range(0, len(trendData)):
            trendData["Trendline"][j] = j+1.5
        trendDataMain = pd.concat([trendDataMain, trendData]).reset_index(drop=True)
    salesData = trendDataMain.copy()
    return salesData


# In[29]:


# Creating a holiday flag to capture effect of holiday on sales whenever a one or more holidays are falling in a week.
def holiday(holidayData, timeData):
    timeData1 = timeData.copy()
    holidayData["Date"] = pd.DatetimeIndex(holidayData[weekVariable])
    holidayData["Year"] = pd.DatetimeIndex(holidayData[weekVariable]).year
    holidayData["Month"] = pd.DatetimeIndex(holidayData[weekVariable]).month
    holidayData = holidayData.merge(timeData1, how='left', on=weekVariable)
    holidayData["Holiday Flag"] = 1
    holidayData = holidayData[[versionVariable, locationCountryVariable, 'Year', 'Month', 'Week', 'Holiday Flag']].drop_duplicates()
    return holidayData


# In[30]:


# Creating a flag to capture effect of power date on sales whenever a one or more power date are falling in a week.
def power_date(powerDateData, timeData):
    timeData1 = timeData.copy()
    powerDateData["Date"] = pd.DatetimeIndex(powerDateData[dayVariable])
    powerDateData["Year"] = pd.DatetimeIndex(powerDateData[dayVariable]).year
    powerDateData["Month"] = pd.DatetimeIndex(powerDateData[dayVariable]).month
    powerDateData = powerDateData.merge(timeData1, how='left', on=weekVariable)
    powerDateData["Power Date Flag"] = 1
    powerDateData = powerDateData[[versionVariable, locationCountryVariable, rgm5Variable, 'Year', 'Month', 'Week', 'Power Date Flag']].drop_duplicates()
    return powerDateData


# In[31]:


# Creating Year and Month columns using 'Time.[Planning Month]' column. 
def planning_month_year(data, centuryFirstTwoInitial = '20'):
    data['Year'] = data[planningMonthVariable].apply(lambda x: int(centuryFirstTwoInitial + x.split('-')[1]))
    data['Month'] = data[planningMonthVariable].apply(lambda x: int(x.split('-')[0][1:]) if '0' not in x.split('-')[0][1:2] else int(x.split('-')[0][2:]))
    return data


# In[32]:


def day_to_week(data, timeData):
    timeData1 = timeData.copy()
    data['Date'] = pd.DatetimeIndex(data[dayVariable])
    data['Year'] = pd.DatetimeIndex(data[dayVariable]).year
    data['Month'] = pd.DatetimeIndex(data[dayVariable]).month
    timeData1.rename(columns={weekVariable:dayVariable}, inplace=True)
    data = data.merge(timeData1, how='left', on=dayVariable)
    return data


# In[33]:


def daily_level_external_factors(countryName, listOfDailyLevelExternalData, timeData):
    externalDict = {}
    count = 1
    for i in listOfDailyLevelExternalData:
        name = str('externalDataFrame' + countryName + str(count))
        externalDataFrame = pd.DataFrame()
        externalDataFrame = day_to_week(i, timeData)
        externalDataFrame = externalDataFrame.drop(columns=['Date', dayVariable])
        externalDataFrame = externalDataFrame.groupby(['Year', 'Month', 'Week', versionVariable, marketIntelCountryVariable]).mean().reset_index()
        externalDataFrame.rename(columns={marketIntelCountryVariable:locationCountryVariable}, inplace=True)
        externalDataFrame = externalDataFrame[externalDataFrame[locationCountryVariable]==countryName]
        externalDict[name] = externalDataFrame
        count = count + 1
    return externalDict


# In[34]:


def weekly_level_external_factors(countryName, listOfWeeklyLevelExternalData, timeData):
    timeData1 = timeData.copy()
    externalDict = {}
    count = 1
    for i in listOfWeeklyLevelExternalData:
        name = str('externalDataFrame' + countryName + str(count))
        externalDataFrame = i.copy()
        externalDataFrame['Date'] = pd.DatetimeIndex(externalDataFrame[weekVariable])
        externalDataFrame['Year'] = pd.DatetimeIndex(externalDataFrame[weekVariable]).year
        externalDataFrame['Month'] = pd.DatetimeIndex(externalDataFrame[weekVariable]).month
        externalDataFrame = externalDataFrame.merge(timeData1, how='left', on=weekVariable)
        externalDataFrame = externalDataFrame.drop(columns=['Date', weekVariable])
        externalDataFrame.rename(columns={marketIntelCountryVariable:locationCountryVariable}, inplace=True)
        externalDataFrame = externalDataFrame[externalDataFrame[locationCountryVariable]==countryName]
        externalDict[name] = externalDataFrame
        count = count + 1
    return externalDict


# In[35]:


def monthly_level_macroeconomic(countryName, listOfMonthlyLevelMacroeconomicData):
    macroeconomicDict = {}
    count = 1
    for i in listOfMonthlyLevelMacroeconomicData:
        name = str('macroeconomicDataFrame' + countryName + str(count))
        macroeconomicDataFrame = pd.DataFrame()
        macroeconomicDataFrame = planning_month_year(i)
        macroeconomicDataFrame = macroeconomicDataFrame.drop(columns=[planningMonthVariable])
        macroeconomicDataFrame.rename(columns={marketIntelCountryVariable:locationCountryVariable}, inplace=True)
        macroeconomicDataFrame = macroeconomicDataFrame[macroeconomicDataFrame[locationCountryVariable]==countryName]
        macroeconomicDict[name] = macroeconomicDataFrame
        count = count + 1
    return macroeconomicDict


# In[36]:


def loyalty(sellOutData, loyaltyData):
    mergeGranularity = [i for i in loyaltyData.columns if i in sellOutData.columns]
    sellOutAndLoyaltyData = sellOutData.merge(loyaltyData, how='left', on=mergeGranularity)
    loyaltyDataStructured = sellOutAndLoyaltyData.groupby(by=sellOutDataGranularityGroupBy)["RGM Loyalty"].mean().reset_index()
    return loyaltyDataStructured


# In[37]:


# Here we are calculating seasonality index by taking ratio of weekly average of sales for all year to overall average of sales.
def seasonality_index(salesData, listOfSeasonalityYear):
    data = salesData.groupby(by=['Year', seasonalityTimeGranularity, rgm5Variable, locationCountryVariable, channelVariable])[dependentVariable].sum().reset_index()
    data = data[data['Year'].isin(listOfSeasonalityYear)].reset_index(drop=True)
    data['Key'] = data[rgm5Variable] + data[locationCountryVariable] + data[channelVariable]
    data = data.pivot_table(index=[seasonalityTimeGranularity, rgm5Variable, locationCountryVariable, channelVariable, 'Key'], columns='Year', values=dependentVariable, aggfunc=np.sum).reset_index().fillna(0)
    data = data.rename_axis(None, axis=1)
    seasonalityDataMain = pd.DataFrame()
    for i in data["Key"].unique():
        seasonalityData = data[data["Key"]==i].reset_index(drop=True)
        seasonalityDataCount = seasonalityData.iloc[:,len(seasonalityData.columns)-2:len(seasonalityData.columns)]
        seasonalityDataCount['Count'] = np.count_nonzero(seasonalityDataCount, axis=1)
        seasonalityData['Count'] = seasonalityDataCount['Count']
        if len(listOfSeasonalityYear) == 1:
            seasonalityData["Weekly Average"] = (seasonalityData[listOfSeasonalityYear[0]])/seasonalityData['Count']
        elif len(listOfSeasonalityYear) == 2:
            seasonalityData["Weekly Average"] = (seasonalityData[listOfSeasonalityYear[0]] + seasonalityData[listOfSeasonalityYear[1]])/seasonalityData['Count']
        elif len(listOfSeasonalityYear) == 3:
            seasonalityData["Weekly Average"] = (seasonalityData[listOfSeasonalityYear[0]] + seasonalityData[listOfSeasonalityYear[1]] + seasonalityData[listOfSeasonalityYear[2]])/seasonalityData['Count']
        elif len(listOfSeasonalityYear) == 4:
            seasonalityData["Weekly Average"] = (seasonalityData[listOfSeasonalityYear[0]] + seasonalityData[listOfSeasonalityYear[1]] + seasonalityData[listOfSeasonalityYear[2]] + seasonalityData[listOfSeasonalityYear[3]])/seasonalityData['Count']
        elif len(listOfSeasonalityYear) == 5:
            seasonalityData["Weekly Average"] = (seasonalityData[listOfSeasonalityYear[0]] + seasonalityData[listOfSeasonalityYear[1]] + seasonalityData[listOfSeasonalityYear[2]] + seasonalityData[listOfSeasonalityYear[3]] + seasonalityData[listOfSeasonalityYear[4]])/seasonalityData['Count']
        elif len(listOfSeasonalityYear) == 6:
            seasonalityData["Weekly Average"] = (seasonalityData[listOfSeasonalityYear[0]] + seasonalityData[listOfSeasonalityYear[1]] + seasonalityData[listOfSeasonalityYear[2]] + seasonalityData[listOfSeasonalityYear[3]] + seasonalityData[listOfSeasonalityYear[4]] + seasonalityData[listOfSeasonalityYear[5]])/seasonalityData['Count']
        else:
            logger.debug('Number of seasonality years should be less than or equal to 6.')
        seasonalityData["Overall Average"] = np.average(seasonalityData["Weekly Average"])
        seasonalityData["Seasonality Index"] = seasonalityData["Weekly Average"]/seasonalityData["Overall Average"]
        seasonalityDataMain = pd.concat([seasonalityDataMain, seasonalityData]).reset_index(drop=True)
        seasonalityDataMain.drop(columns=['Key', 'Weekly Average', 'Overall Average', 'Count']+listOfSeasonalityYear, inplace=True)
    return seasonalityDataMain


# In[38]:


def adstock(currentValue, previousValue, decayRate):
    pass


# In[39]:


# Calculating correlation between each and every column.
def correlation_check(data):
    data = data.drop(columns=['Year', 'Month', 'Week'])
    correlation_data = data.corr(method ='pearson')
    return correlation_data


# In[40]:


# Calculating vif between model input variables to avoid multi-collinearity.
def vif_check(data, listOfInputVariablesForModel):
    X = data[listOfInputVariablesForModel]
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
    return vif_data


# In[41]:


# Merging output of an individual methods to prepare sales data.
def sales_data_prep():
    salesData = discount_index(sellOutData)
    distributionData = distribution(sellOutData)
    salesData = salesData.merge(distributionData, how='inner', on=sellOutDataGranularityGroupBy).reset_index(drop = True)
    salesData = pantry_loading(salesData)
    return salesData


# In[42]:


# Running individual methods to create master data for modelling.
def master_data_prep():
    salesData = sales_data_prep()
    logger.debug("Successfully derived variables for weighted distribution, price, discount & pantry loading.")
    
    salesData = price_threshold(salesData)
    salesData = trend_line(salesData)
    seasonalityIndex = seasonality_index(salesData, listOfSeasonalityYear)
    salesData = salesData.merge(seasonalityIndex, how='left', on=[seasonalityTimeGranularity, rgm5Variable, locationCountryVariable, channelVariable])
    logger.debug("Successfully derived variables for trendline & seasonality index.")
    
    try:
        promoMechanicAndDisplayData = prepare_promo_data(sellOutData, tradePromoCalendarData, timeData)
        salesData = salesData.merge(promoMechanicAndDisplayData, how='left', on=sellOutDataGranularityGroupBy).reset_index(drop=True)
    except:
        logger.debug("Warning: Promo data is missing.")
        
    try:
        loyaltyDataStructured = loyalty(sellOutData, loyaltyData)
        salesData = salesData.merge(loyaltyDataStructured, how='left', on=sellOutDataGranularityGroupBy).reset_index(drop=True)
    except:
        logger.debug("Warning: Loyalty data is missing.")
    
    salesDataAndExternalData = salesData[~salesData[locationCountryVariable].isin(list(dictOfDailyLevelExternalData.keys()))]
    for countryName, externalData in dictOfDailyLevelExternalData.items():
        salesDataDummy = salesData[salesData[locationCountryVariable]==countryName]
        externalDict1 = {}
        countryName = countryName
        listOfDailyLevelExternalData = externalData
        if len(listOfDailyLevelExternalData)!=0:
            externalDict0 = daily_level_external_factors(countryName, listOfDailyLevelExternalData, timeData)
            externalDict1.update(externalDict0)
            for i in (list(externalDict1.keys())):
                externalDataFrame = externalDict1[i]
                salesDataDummy = salesDataDummy.merge(externalDataFrame, how='left', on=['Year', 'Month', 'Week', versionVariable, locationCountryVariable])
        else:
            logger.debug("Warning: Daily level external data is missing for " + countryName + ".")
        salesDataAndExternalData = pd.concat([salesDataAndExternalData, salesDataDummy], axis=0).reset_index(drop=True)
    salesData = salesDataAndExternalData.copy()
    
    salesDataAndExternalData = salesData[~salesData[locationCountryVariable].isin(list(dictOfWeeklyLevelExternalData.keys()))]
    for countryName, externalData in dictOfWeeklyLevelExternalData.items():
        salesDataDummy = salesData[salesData[locationCountryVariable]==countryName]
        externalDict1 = {}
        countryName = countryName
        listOfWeeklyLevelExternalData = externalData
        if len(listOfWeeklyLevelExternalData)!=0:
            externalDict0 = weekly_level_external_factors(countryName, listOfWeeklyLevelExternalData, timeData)
            externalDict1.update(externalDict0)
            for i in (list(externalDict1.keys())):
                externalDataFrame = externalDict1[i]
                salesDataDummy = salesDataDummy.merge(externalDataFrame, how='left', on=['Year', 'Month', 'Week', versionVariable, locationCountryVariable])
        else:
            logger.debug("Warning: Weekly level external data is missing for " + countryName + ".")
        salesDataAndExternalData = pd.concat([salesDataAndExternalData, salesDataDummy], axis=0).reset_index(drop=True)
    salesData = salesDataAndExternalData.copy()
    
    salesDataAndExternalData = salesData[~salesData[locationCountryVariable].isin(list(dictOfMonthlyLevelExternalData.keys()))]
    for countryName, externalData in dictOfMonthlyLevelExternalData.items():
        salesDataDummy = salesData[salesData[locationCountryVariable]==countryName]
        macroeconomicDict1 = {}
        countryName = countryName
        listOfMonthlyLevelMacroeconomicData = externalData
        if len(listOfMonthlyLevelMacroeconomicData)!=0:
            macroeconomicDict0 = monthly_level_macroeconomic(countryName, listOfMonthlyLevelMacroeconomicData)
            macroeconomicDict1.update(macroeconomicDict0)
            for i in (list(macroeconomicDict1.keys())):
                externalDataFrame = macroeconomicDict1[i]
                salesDataDummy = salesDataDummy.merge(externalDataFrame, how='left', on=['Year', 'Month', versionVariable, locationCountryVariable])
        else:
            logger.debug("Warning: Monthly level external data is missing for " + countryName + ".")
        salesDataAndExternalData = pd.concat([salesDataAndExternalData, salesDataDummy], axis=0).reset_index(drop=True)
    salesData = salesDataAndExternalData.copy()
    
    salesData.drop(columns = "Granularity", inplace=True)
    
    try:
        holidayData1 = holiday(holidayData, timeData)
        salesData = salesData.merge(holidayData1, how='left', on=[versionVariable, locationCountryVariable, 'Year', 'Month', 'Week']).fillna(0)
    except:
        logger.debug("Warning: Holiday data is missing.")
    
    try:
        powerDateData1 = power_date(powerDateData, timeData)
        salesData = salesData.merge(powerDateData1, how='left', on=[versionVariable, locationCountryVariable, rgm5Variable, 'Year', 'Month', 'Week']).fillna(0)
    except:
        logger.debug("Warning: Power date data is missing.")
    
    masterData = salesData.copy()
    masterData = masterData.reset_index(drop=True)
    masterData.fillna(0, inplace=True)
    return masterData


# In[43]:


sellOutData = fill_missing_rows_between_start_and_end_date_of_sellOut_data(sellOutData)
masterData = master_data_prep()


# In[44]:


logger.debug("Successfully prepared the master data.")


# In[45]:


# Converting master data from wide format to long format.
masterData = pd.melt(masterData, id_vars=sellOutDataGranularityGroupBy, value_vars=[i for i in masterData.columns if i not in sellOutDataGranularityGroupBy], var_name='Promo Simulation.[Promo Variables]', value_name='RGM Promo Simulation Script 1 Output Placeholder')


# In[46]:


masterData = masterData.merge(timeDataOriginal, how='left', on=['Year', 'Month', 'Week']).drop(columns=['Year', 'Month', 'Week'])


# In[47]:


masterDataAudit = masterData.copy()


# In[48]:


masterData = masterData[[versionVariable, countryVariable, regionVariable, 
                         locationCountryVariable, locationVariable, 
                         channelVariable, customerGroupVariable, 
                         rgm1Variable, rgm2Variable, rgm5Variable, rgm6Variable, rgm7Variable, ppgVariable, weekVariable, 
                         'Promo Simulation.[Promo Variables]', 'RGM Promo Simulation Script 1 Output Placeholder']]


# In[49]:


# Preparing data for audit table.

# Create an empty prepareCurrentAuditDataFrame with header names.
auditColumns = [versionVariable, seriesVariable, dayVariable, modelRunTypeVariableName, pluginVariableName, machineLearningAlgorithmVariableName, dataStartDateVariableName, dataEndDateVariableName, pluginExecutionTimeStampVariableName, pluginExecutionTimeVariableName]
prepareCurrentAuditDataFrame = pd.DataFrame(columns = auditColumns)

# Get the last date for which we processed the sellout data from the audit table.
try:
    if modelRunType in ['refreshWithoutModelRun', 'refreshWithModelRun']:
        auditDataFrame1 = auditDataFrame[(auditDataFrame[pluginVariableName]=='Postprocessing')]
        auditDataFrame2 = auditDataFrame1[(pd.to_datetime(auditDataFrame1[pluginExecutionTimeStampVariableName])==pd.to_datetime(auditDataFrame1[pluginExecutionTimeStampVariableName]).max())]
        lastProcessedDate = pd.to_datetime(auditDataFrame2[dataEndDateVariableName]).max().strftime("%d-%b-%y")
except:
    if auditDataFrame.empty:
        modelRunType = 'freshModelRun'

if modelRunType in ['freshModelRun']:
    prepareCurrentAuditDataFrame[dataStartDateVariableName] = [pd.to_datetime(masterDataAudit[weekVariable]).min().strftime("%d-%b-%y")]
    prepareCurrentAuditDataFrame[dataEndDateVariableName] = [pd.to_datetime(masterDataAudit[weekVariable]).max().strftime("%d-%b-%y")]
elif modelRunType in ['refreshWithoutModelRun', 'refreshWithModelRun']:
    try:
        prepareCurrentAuditDataFrame[dataStartDateVariableName] = [pd.to_datetime(masterDataAudit[(pd.to_datetime(masterDataAudit[weekVariable]) > pd.to_datetime(lastProcessedDate))][weekVariable]).min().strftime("%d-%b-%y")]
    except:
        prepareCurrentAuditDataFrame[dataStartDateVariableName] = [pd.to_datetime(masterDataAudit[(pd.to_datetime(masterDataAudit[weekVariable]) >= pd.to_datetime(lastProcessedDate))][weekVariable]).min().strftime("%d-%b-%y")]
    try:
        prepareCurrentAuditDataFrame[dataEndDateVariableName] = [pd.to_datetime(masterDataAudit[(pd.to_datetime(masterDataAudit[weekVariable]) > pd.to_datetime(lastProcessedDate))][weekVariable]).max().strftime("%d-%b-%y")]
    except:
        prepareCurrentAuditDataFrame[dataEndDateVariableName] = [pd.to_datetime(masterDataAudit[(pd.to_datetime(masterDataAudit[weekVariable]) >= pd.to_datetime(lastProcessedDate))][weekVariable]).max().strftime("%d-%b-%y")]

        
# Get current timestamp and duration of plugin execution.
now = datetime.datetime.now()
currTimestamp = now.strftime("%Y-%m-%d %H:%M:%S")
endTime = time.time()

prepareCurrentAuditDataFrame[versionVariable] = [masterDataAudit[versionVariable].unique()[0]]
prepareCurrentAuditDataFrame[seriesVariable] = [int(tenantSequenceID[sequenceId].loc[0])]
prepareCurrentAuditDataFrame[dayVariable] = [tenantCurrDay[dayVariable].loc[0]]

prepareCurrentAuditDataFrame[pluginVariableName] = ['Preprocessing']
# prepareCurrentAuditDataFrame[machineLearningAlgorithmVariableName] = [modelName] 
prepareCurrentAuditDataFrame[modelRunTypeVariableName] = [modelRunType]  # Either 'freshModelRun', 'refreshWithoutModelRun', 'refreshWithModelRun'
prepareCurrentAuditDataFrame[pluginExecutionTimeStampVariableName] = [pd.to_datetime(currTimestamp)] # Get current timestamp
pluginExecutionTimeInMinutes = np.round((endTime-startTime)/60,2)
prepareCurrentAuditDataFrame[pluginExecutionTimeVariableName] = [pluginExecutionTimeInMinutes]  # Get duration of plugin execution

auditDataFrame = auditDataFrame.append(prepareCurrentAuditDataFrame, ignore_index=True)
auditDataFrameCopy = auditDataFrame.copy()


# In[50]:


## Output for Tenant
## Uncomment When Required

#UserStory:212123 - channelVariable is one of the least granual under Channel dimension so added in all the output
masterData = masterData[[versionVariable,channelVariable, locationVariable, customerGroupVariable, ppgVariable, weekVariable,
                         'Promo Simulation.[Promo Variables]', 'RGM Promo Simulation Script 1 Output Placeholder']]


# In[51]:


logger.debug("For masterData, shape is " + str(masterData.shape))
logger.debug("For auditDataFrameCopy, shape is " + str(auditDataFrameCopy.shape))
logger.debug("Pre-processing code for promo model is ran successfully and output is saved.")


# In[52]:


# masterData.to_csv('Master Data.csv', index=False)
# auditDataFrameCopy.to_csv('auditTableDataOutputFinal.csv', index=False)


